This is a work in progress :)

In [28]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import matplotlib
matplotlib.use('Agg')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory


from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [29]:
df = pd.read_csv('../input/Tweets.csv')

In [30]:
df.head()

In [31]:
positive = df[df['airline_sentiment']=='positive']
negative = df[df['airline_sentiment']=='negative']
nr_group = negative.groupby(['negativereason',"airline"],as_index=False).count()
negative['negativereason'].unique()

In [32]:
negative['airline'].unique()

In [33]:
import matplotlib.pyplot as plt
import seaborn as sns
f,axarray = plt.subplots(1,2,figsize=(15,10))
g = sns.barplot(x= nr_group['negativereason'],y=nr_group['tweet_id'],ax=axarray[0])
for tick in axarray[0].get_xticklabels():
        tick.set_rotation(90)
sns.barplot(x= nr_group['airline'],y=nr_group['tweet_id'],ax=axarray[1])
for tick in axarray[1].get_xticklabels():
        tick.set_rotation(90)
axarray[0].xaxis.set_label_position('top') 
axarray[1].xaxis.set_label_position('top') 

Removing @ which mostly refers to the airline and does not add information.

In [34]:
import re 
text = [re.sub(r"@(\S*)",r"",text) for text in df['text'].astype('str')]

In [35]:
text[0:5]

Removing URLs

In [36]:
text = [re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'," ",t) for t in text]

Recognize Emoticons and transform them into text. For inscante turn :) into 'Happy'.

In [37]:
#happyness
text = [re.sub(r':\)|:-\)|:D|;\)|;-\)|:-D',"HAPPY",t) for t in text]
#sadness
text = [re.sub(r':\(|:-\(|;\(|;-\(',"SAD",t) for t in text]


In [38]:
text[0:5]

In [39]:
import unicodedata
#text = [re.sub('u[\U0001F602-\U0001F64F]', lambda m: unicodedata.name(m.group()), t,flags=re.UNICODE) for t in text]

In [40]:
text = [re.sub("[^a-zA-Z]"," ",t).lower() for t in text]


In [41]:
text = pd.Series(text)

## Bag of Words

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
                             max_features = 5000) 
tdf = vectorizer.fit_transform(text)


In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators = 20) 
scores = cross_val_score(clf, tdf.toarray(), df["airline_sentiment"], cv=5)
scores.mean()

## Tri-Grams

Let's now try using 3-grams which ar sequences of 3 words. 

In [17]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,
                             ngram_range=(3,3),   
                             max_features = 5000) 
tdf = vectorizer.fit_transform(text) 

In [18]:
clf = RandomForestClassifier(n_estimators = 20) 
scores = cross_val_score(clf, tdf.toarray(), df["airline_sentiment"], cv=5)
scores.mean()

## NLTK

I am now going to exploit a Natural Language processing library, called NLTK ( Bird, Steven, Edward Loper and Ewan Klein (2009), Natural Language Processing with Python. O’Reilly Media Inc.) to extract some valuable information from tweet I hope to use to improve my models.

So far our tweets are still strings, without weird symbols and punctuations but strings. The first thing we are going to do is to tokenize them  which means we are going to create lists of words.

In [42]:
text.head()

In [43]:
import nltk
tokens = [nltk.word_tokenize(t) for t in text]

In [44]:
tokens[0:2]

Nowe we are going to pos tag each tweet. Pos Tagging, which stands for Part Of Speech Tagging is the process through which each word of a sentence is paired with its role (Noun, Verb, Adjective, etc.) in that sentence. https://en.wikipedia.org/wiki/Part-of-speech_tagging

In [45]:
tags = [nltk.pos_tag(t) for t in tokens]

In [46]:
tags[4]

In [129]:
adjectives = []
for i in range(len(tags)):
    tmp = []
    for j in range(len(tags[i])):
        if tags[i][j][1]=='JJ' or tags[i][j][1]== 'JJS' or tags[i][j][1]=='JJR' or tags[i][j][1]=='VBG' or tags[i][j][1]== 'VBG' or tags[i][j][1]=='VBD':
            tmp.append(tags[i][j][0])
    adjectives.append(tmp)

In [130]:
adjectives[0:5]

In [131]:
adjectives = [" ".join(adj) for adj in adjectives]

In [132]:
from sklearn.feature_extraction.text import CountVectorizer

adj_vec = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,
                             ngram_range=(1,1),   
                             max_features = 5000) 
adf = adj_vec.fit_transform(adjectives) 

In [133]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators = 10) 
scores = cross_val_score(clf, adf.toarray(), df["airline_sentiment"], cv=5)
scores.mean()